# Example: Vector Hashing of Sarcasm Samples
Fill me in

## Setup
We set up the computational environment by including [the `Include. jl` file](Include.jl) using [the `include(...)` method](https://docs.julialang.org/en/v1/base/base/#Base.include). The [`Include.jl` file](Include.jl) loads external packages and functions we will use in these examples. 
* For additional information on functions and types used in this example, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

In [3]:
include("Include.jl");

## Prerequisites
Fill me in

In [5]:
path_to_save_file = joinpath(_PATH_TO_DATA, "L4a-SarcasmSamplesTokenizer-SavedData.jld2");

Fill me in

In [7]:
saved_data_dictionary = load(path_to_save_file);
tokendictionary = saved_data_dictionary["tokendictionary"];
dataset = saved_data_dictionary["dataset"];
number_of_records = saved_data_dictionary["number_of_records"];

In [8]:
number_of_tokens = tokendictionary |> length;

In [9]:
dataset.records

Dict{Int64, MySarcasmRecordModel} with 28619 entries:
  19700 => MySarcasmRecordModel(false, "in memoriam robin thickes career", "htt…
  21664 => MySarcasmRecordModel(false, "security video pokes holes in robbers t…
  11950 => MySarcasmRecordModel(true, "author dismayed by amazon customers othe…
  1703  => MySarcasmRecordModel(true, "dollhousing crisis set to worsen mean ol…
  12427 => MySarcasmRecordModel(false, "muslims respond to hateful protests wit…
  7685  => MySarcasmRecordModel(false, "this teens trying to make the road safe…
  18374 => MySarcasmRecordModel(true, "fear notshe means you no harm says eliza…
  3406  => MySarcasmRecordModel(false, "great lakes amazing connections the pow…
  23970 => MySarcasmRecordModel(true, "clinton credits nevada victory to inesca…
  27640 => MySarcasmRecordModel(true, "merger of advertising giants brings toge…
  28576 => MySarcasmRecordModel(true, "bartender refuses to acknowledge patrons…
  1090  => MySarcasmRecordModel(false, "clinton announc

## Task 1: Compute the Hashvectors for Sarcastic Samples
Fill me in

In [11]:
sarcasim_hashed_dictionary = Dict{Int64, Array{Int64,1}}();
for i ∈ 1:number_of_records
    record = dataset.records[i];
    is_sarcastic_flag = record.issarcastic
    if (is_sarcastic_flag == true)
        
        # split -
        fields = split(record.headline, ' ') .|> String        
        sarcasim_hashed_dictionary[i] = hashing(fields, hash = tokendictionary, size = (number_of_tokens+1));
    end
end

In [23]:
findall(x -> x != 0, sarcasim_hashed_dictionary[1])

8-element Vector{Int64}:
  5552
  8294
 12046
 15827
 18532
 23294
 26616
 27979